In [13]:
# 파일 만들어서 별도 프로세스에서 실행하는 버전
import os
import sys
import subprocess
import json
from typing import Dict, List, Optional, Union


# 상수 정의
BATCH_SIZE = 3
TIMEOUT_SECONDS = 180


def run_crawler_in_separate_process(search_keyword: str) -> bool:
    """별도의 Python 프로세스에서 크롤러를 실행하는 함수"""
    script_content = _generate_crawler_script(search_keyword)
    script_path = "temp_crawler_script.py"
    
    try:
        _write_script_file(script_path, script_content)
        return _execute_crawler_script(script_path)
    except Exception as e:
        print(f"크롤링 실행 중 오류 발생: {e}")
        return False
    finally:
        _cleanup_script_file(script_path)

def _generate_crawler_script(search_keyword: str) -> str:
    """크롤러 스크립트 내용을 생성"""
    return f'''
import sys
import os
sys.path.append(os.path.dirname(os.path.abspath(__file__)))

from scrapy.crawler import CrawlerProcess
from danawa_crawler.spiders.product_spider import ProductSpider
from scrapy.utils.project import get_project_settings

def main():
    settings = get_project_settings()
    process = CrawlerProcess(settings=settings)
    process.crawl(ProductSpider, search_keyword="{search_keyword}")
    process.start()

if __name__ == "__main__":
    main()
'''

def _write_script_file(script_path: str, content: str) -> None:
    """스크립트 파일을 생성"""
    with open(script_path, 'w', encoding='utf-8') as f:
        f.write(content)

def _execute_crawler_script(script_path: str) -> bool:
    """크롤러 스크립트를 실행"""
    result = subprocess.run(
        [sys.executable, script_path], 
        capture_output=True, 
        text=True, 
        timeout=TIMEOUT_SECONDS
    )
    
    if result.returncode == 0:
        print("크롤링이 성공적으로 완료되었습니다!")
        return True
    else:
        print(f"크롤링 중 오류: {result.stderr}")
        return False

def _cleanup_script_file(script_path: str) -> None:
    """임시 스크립트 파일을 삭제"""
    if os.path.exists(script_path):
        os.remove(script_path)

def load_product_data(search_keyword: str) -> dict:
    """JSON 파일에서 상품 데이터를 로드"""
    file_path = f"{search_keyword}_nutrition_data.json"
    
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        return data
    except FileNotFoundError:
        raise FileNotFoundError(f"파일을 찾을 수 없습니다: {file_path}")


def get_target_product(search_keyword: str) -> dict | None:
    """검색 키워드에 해당하는 목표 상품을 찾아 반환"""
    try:
        data = load_product_data(search_keyword)
    except (FileNotFoundError) as e:
        print(f"데이터 로드 오류: {e}")
        return None
    
    if len(data) == 0:
        print("검색 결과가 없습니다.")
        return None
    
    # 첫 번째 상품 시도
    first_product = data[0]
    print("첫 번째 상품:")
    print(first_product['prod_name'])
    
    # 사용자 선택 (테스트용으로 'y'로 설정)
    user_agreement = 'n'  # 실제: input("이 상품으로 진행하시겠습니까? (y/n): ") < steamlit에서 입력받는 방식으로 변경
    
    if user_agreement.lower() == 'y':
        return first_product
    
    # 첫 번째 상품이 아닌 경우, 배치 단위로 상품 표시
    return _process_alternative_products(data)

def _process_alternative_products(data: dict) -> dict | None:
    """대안 상품들을 배치 단위로 처리"""

    for start_idx in range(1, len(data), BATCH_SIZE):
        products_info = []
        
        batch_num = 1
        for idx in range(start_idx, min(start_idx + BATCH_SIZE, len(data))):  # 배치 크기 만큼 상품 표시 (현재는 3개)
            products_info.append(data[idx])
            print(f"{batch_num}번 제품: \n {products_info[batch_num-1]}") # 배치 크기 만큼 저장된 상품 정보 데이터 전송
            batch_num += 1
    
        user_selection = 2 # 실제: input("1 ~ 3번 제품 중 선택하세요 (선택 안함: n): ") < steamlit에서 입력받는 방식으로 변경
        
        if user_selection != 'n' and 1 <= user_selection <= len(products_info):
            return products_info[user_selection - 1]
        else:
            products_info = [] # 배치 초기화
            batch_num = 1 
            continue

    print("선택된 상품이 없습니다.")
    return None

def main() -> int:
    """메인 함수"""
    search_keyword = "제로 콜라" # ocr로 추출한 텍스트 데이터 입력
    
    # 크롤링 실행 (현재는 주석 처리)
    # run_crawler_in_separate_process(search_keyword)
    
    # 목표 상품 찾기
    target_data = get_target_product(search_keyword)
    
    if target_data:
        print("\n최종 선택된 상품:")
        print(target_data)
    else:
        print("상품을 찾을 수 없습니다.")
    
    return 0

if __name__ == "__main__":
    main()

첫 번째 상품:
코카 콜라 음료 코카 콜라 제로 300ml
1번 제품: 
 {'item_num': 2, 'prod_name': '일화 부르르 제로 콜라 1.5L', 'spec_origin': '탄산음료 / 콜라 / 포장형태: 페트 / 제로 칼로리 / 박스 / [영양정보] 1회 제공량 100ml당 열량: 0kcal / 탄수화물: 0g / 당류: 0g / 단백질: 0g / 지방: 0g / 포화지방: 0g / 트랜스지방: 0g / 콜레스테롤: 0mg / 나트륨: 5mg', 'price': '16650', 'per_serving': '100ml', 'kcal': '0', 'carb': '0', 'sugar': '0', 'protein': '0', 'fat': '0'}
2번 제품: 
 {'item_num': 3, 'prod_name': '코카 콜라 음료 코카 콜라 제로 250ml', 'spec_origin': '탄산음료 / 콜라 / 포장형태: 캔 / 제로 칼로리 / 박스 / [영양정보] 1회 제공량 250ml당 열량: 0kcal / 탄수화물: 0g / 당류: 0g / 단백질: 0g / 지방: 0g / 포화지방: 0g / 트랜스지방: 0g / 콜레스테롤: 0mg / 나트륨: 19mg', 'price': '880', 'per_serving': '250ml', 'kcal': '0', 'carb': '0', 'sugar': '0', 'protein': '0', 'fat': '0'}
3번 제품: 
 {'item_num': 4, 'prod_name': '코카 콜라 음료 스프라이트 제로 500ml', 'spec_origin': '탄산음료 / 사이다 / 포장형태: 페트 / 제로 칼로리 / [영양정보] 총 내용량 500ml당 열량: 0kcal / 탄수화물: 0.2g / 당류: 0g / 단백질: 0g / 지방: 0g / 포화지방: 0g / 트랜스지방: 0g / 콜레스테롤: 0mg / 나트륨: 3mg', 'price': '800', 'per_serving': '', 'kcal': '0',